Get the pathways


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

h=[]
t=[]
hid=[]
all_href_list = []
for page in range(1,20):
    print(page)
    chrome_options = Options()
    prefs = {
        "profile.managed_default_content_settings.stylesheets": 2,
        "profile.managed_default_content_settings.images": 2
    }
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options = Options()
    chrome_options.add_argument("--headless")  
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(3600)
    
    url = f'https://smpdb.ca/view?export=true&limit=50&page={page}&pathbank=true&sort_by=rank&sort_order=asc&species_id=1&species_name=Homo+sapiens'
    driver.get(url)
    if True:
        try:
            tbody_element = driver.find_element(By.TAG_NAME, 'tbody')
            h2_elements = tbody_element.find_elements(By.TAG_NAME, 'h2')
            elements_with_type = driver.find_elements(By.CSS_SELECTOR, '[class*="type"]')
            div_elements = driver.find_elements(By.CLASS_NAME, 'path-description')
            ul_elements = driver.find_elements(By.CLASS_NAME, 'browse-list')
            visible_divs = [div for div in div_elements if 'display: none' in div.get_attribute('style')]
            for i in h2_elements:
                h.append(i.text)
            for i in elements_with_type:
                t.append(i.text)
            for i in visible_divs:
                element_text = driver.execute_script("return arguments[0].innerText;", i)
                hid.append(element_text)
        except:
            pass

    
        i=0
        for ul in ul_elements:
            try:
                i+=1
                li_elements = ul.find_elements(By.TAG_NAME, 'li')
                href_list = [li.find_element(By.TAG_NAME, 'a').get_attribute('href') for li in li_elements][:-1]
                if i%2!=0:
                    href_list_id = [i[-11:] for i in href_list]
                else:
                    href_list_id = [i[-6:] for i in href_list]
                all_href_list.append(href_list_id)
            except:
                pass
        else:
            print("No <tbody> tag found on the page.")
    driver.quit()


Create pathways json

In [ ]:
ls=[]
for i in range(len(h)):
    if i%2==0:
        try:
            temp={}
            temp['smpdb'] = h[i]
            temp['name']=h[i+1]
            temp['class']=t[i//2]
            temp['description']=hid[i//2]
            temp['chem_link']=all_href_list[i]
            temp['prot_link']=all_href_list[i+1]
            ls.append(temp)
        except:
            print(i)

In [3]:
import json
for i in ls:
    filename=f'pathway/{i["smpdb"]}.json'
    with open(filename,'w') as file:
        json.dump(i,file,indent=4)

Process links for chemicals and proteins

In [4]:
links=[]
for i in all_href_list:
    links.extend(i)

In [5]:
links = set(links)
print(len(links), ' links found')

3277  links found


Fetch data for chemicals and proteins

In [ ]:
chem_list=[]
prot_list=[]

chrome_options = Options()
prefs = {
    "profile.managed_default_content_settings.stylesheets": 2,  
    "profile.managed_default_content_settings.images": 2  
}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")
driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(3600)
for i in links:
    try:
        if 'HMDB' in i:
            url = f'https://hmdb.ca/metabolites/{i}'
            driver.get(url)
            strong_elements = driver.find_elements(By.TAG_NAME, "strong")
            name=''
            smiles=''
            WebDriverWait(driver,60).until(
            EC.presence_of_element_located((By.CLASS_NAME, "word-break-all"))
            )
            div_element = driver.find_element(By.CLASS_NAME, "word-break-all")
            smiles = div_element.text
            for element in strong_elements:
                name = element.text
                break
            final_dict={'common_name': name, 'url':url,'smiles':smiles}
            chem_list.append(final_dict)
        else:
            url = f'https://www.uniprot.org/uniprotkb/{i}/entry#sequences'
            driver.get(url)
            WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, "sequence"))
            )
            strong_elements = driver.find_elements(By.TAG_NAME, "strong")
            sequence_div = driver.find_element(By.CLASS_NAME, "sequence")
            span_elements = sequence_div.find_elements(By.TAG_NAME, "span")

            name = ''
            amino_acid=[]
            for div_item in strong_elements:
                name = div_item.text
                break
            for seq in span_elements:
                amino_acid.append(seq.text)
            final_dict = {'name':name,'url':url[:-10],"amino_seq":amino_acid}
            prot_list.append(final_dict)
    except:
        pass
driver.quit()


Create JSON for chemicals and proteins

In [ ]:
for i in chem_list:
    filename=f'chemical/{i["url"][-11:]}.json'
    with open(filename,'w') as file:
        json.dump(i,file,indent=4)
for j in prot_list:
    print(j)
    filename=f'protein/{j["url"][-12:-6]}.json'
    with open(filename,'w') as file:
        json.dump(j,file,indent=4)

            

1754


1523